# Tarea 2 - Aprendizaje bayesiano

### Grupo 36:
     - N. Farías
     - J. M. Varela

## 1. Objetivo

El objetivo de esta tarea es construir un predictor de palabras utilizando el algoritmo Naive Bayes.

Es dificil definir una métrica con la cual cuantificar el éxito del aprendizaje. Por lo tanto, se utilizó la percepción como una forma subjetiva de determinar que la implementación es suficientemente buena, ya que en la mayoría de los casos predice palabras que tienen coherencia con lo escrito previamente.  

Para comparar la calidad de las palabras sugeridas al entrenar el modelo con los distintos hiperparámetros, se utilizó su probabilidad ajustada para tener en cuenta la menor cantidad de términos en los N más bajos. Se plantean dos estrategias distintas para estimar el valor de esos términos faltantes.

## La siguiente celda inicializa los datos de evaluación de rendimiento

In [5]:
probabilidades = {1: {1: [], 2: [], 3: [], 4: []}, 2: {1: [], 2: [], 3: [], 4: []}}

## 2. Diseño

## 2.1 Limpieza de texto

- Para el entrenamiento, se toma como entrada un archivo .txt generado por Whatsapp que contiene los mensajes de un chat grupal.
- Para eliminar la fecha, hora y emisor del mensaje, se considera únicamente el texto después del último ":", si no hay ":" en la línea, se ignora. Esto ocurre en los mensajes que contienen únicamente un archivo multimedia.
- Para todas las frases, ya sean de entrenamiento o ingresadas por el usuario: se pasan a minúscula, se eliminan los signos ",", ";", "?", ".", "!", "-" y los tildes.
- Para el entrenamiento, se cruzan las palabras (luego de las transformaciones anteriores) con un diccionario en español para eliminar las que no estén.

## Ejecutar la siguiente celda para cargar el diccionario

In [1]:
import urllib.request

url = "https://raw.githubusercontent.com/lorenbrichter/Words/master/Words/es.txt"
nombre_archivo_diccionario = "diccionario.txt"

try:
    urllib.request.urlretrieve(url, nombre_archivo_diccionario)
    with open(nombre_archivo_diccionario, 'r') as archivo:
      lineas = archivo.readlines()

    # Elimino el fin de línea con strip()
    diccionario = {x.strip() for x in lineas}
    print(f"El diccionario tiene {len(diccionario)} palabras")
except Exception as e:
    print(f"Ocurrió un error al descargar el archivo de diccionario: {e}")

El diccionario tiene 636598 palabras


## 2.2 Algoritmo

Se implementa el algoritmo Naive Bayes, cuyo entrenamiento consiste en calcular las probabilidades absolutas y condicionales de aparición de las palabras, a partir de la frecuencia y orden con el que aparecen en el documento utilizado para ese fin.

El algoritmo toma como hiperparámetros:
- N: es la cantidad de palabras inmediatamente anteriores a tomar en cuenta para realizar la predicción.
- m: es la cantidad de instancias a agregar para hacer el cálculo del m-estimador, que se utiliza para evitar tener términos nulos cuando se está frente a una instancia no vista durante el entrenamiento.

Algunas consideraciones de diseño sobre la implementación del algoritmo:
- El entrenamiento se realiza por frases. Luego de procesada la frase, se incrementa la variable del modelo donde se lleva el total de palabras procesadas.
- En los diccionarios P y PD, se almacena la cantidad de ocurrencias de las palabras, en lugar de su frecuencia. La frecuencia se calcula con una operación de división en tiempo de predicción. Esto simplifica la operación de reentrenamiento del modelo, dado que al cambiar la cantidad total de palabras (o la cantidad de ocurrencias de una palabra específica) no es necesario recalcular la frecuencia del resto de las palabras.
- La probabilidad p utilizada en la fórmula del m-estimador se calcula como 1 dividido la cantidad de palabras distintas. La cantidad de palabras distintas se calcula con la función len() de Python aplicada al diccionario P, la cual tiene costo computacional de orden constante. Por ese motivo se calcula al momento de recomendar, y consideramos innecesario precalcularla durante el entrenamiento.

## Ejecutar la siguiente celda para inicializar y entrenar el modelo

* Es necesario definir correctamente el **nombre del archivo** que contendrá los chats para el entrenamiento.

* Aquí también se pueden cambiar los **hiperparámetros** del modelo.

In [4]:
#***************************** IMPORTANTE
NOMBRE_ARCHIVO_ENTRENAMIENTO = "chats.txt"

# hiperparámetros

# N define el horizonte de palabras hacia atrás que tomamos en cuenta
N = 3

# m es el utilizado en la fórmula del m-estimador
m = 2

# modelo
cantidad_total_palabras = 0
P = {}
PD = {}


def normalizar_palabra(palabra):
  for punt in [",", ";", "?", ".", "!", "-"]:
    palabra = palabra.replace(punt, "")

  palabra = palabra.lower()

  for k, v in {"á": "a", "é": "e", "í": "i", "ó": "o", "ú": "u"}.items():
    palabra = palabra.replace(k, v)

  return palabra


def entrenar(palabras):
  # preproceso y normalizo las palabras
  palabras = [normalizar_palabra(w) for w in palabras]

  # elimino palabras que no estén en el diccionario
  palabras = [w for w in palabras if w in diccionario]

  for i, palabra in enumerate(palabras):
    P[palabra] = P.get(palabra, 0) + 1
    PD.setdefault(palabra, {})
    previas = set(palabras[max(0, i - N):i])
    for previa in previas:
      PD[palabra][previa] = PD[palabra].get(previa, 0) + 1

  # retorno la cantidad de palabras procesadas
  return len(palabras)


# proceso el archivo de entrenamiento
with open(NOMBRE_ARCHIVO_ENTRENAMIENTO, 'r') as archivo:
  for linea in archivo:
    indice = linea.rfind(":")
    if indice != -1:
      documento = linea[indice + 1:]
      cantidad_total_palabras += entrenar(documento.split())

print(f"{cantidad_total_palabras=}")
print(f"cantidad_palabras_distintas={len(P)}")

cantidad_total_palabras=142272
cantidad_palabras_distintas=11863


## 2.3 Evaluación

- La evaluación de la implementación se realiza analizando la coherencia de la frase obtenida a partir de las predicciones del modelo.
- La comparación de calidad de predicciones entre modelos con distintos hiperparámetros se hace ajustando la probabilidad de la predicción obtenida, para equiparar las probabilidades de N más bajos con las de N más altos, considerando un valor máximo de N=4. El objetivo es que la productoria (sumatoria en nuestro caso por utilizar la función logaritmo) de las probabilidades condicionales P(d|h) tenga siempre la misma cantidad de términos (en este caso 4). De esta manera es como si todas las predicciones se hicieran considerando 4 palabras previas. El desafío está en estimar el valor de los términos faltantes, para lo cual se utilizan dos estrategias:

 - **Estrategia 1**: asumimos que las palabras previas faltantes van a formar parte de las que fueron vistas como previas de h en los datos de entrenamiento. Por lo tanto, al momento de calcular las probabilidades de estas previas ficticias, estimamos el valor "e" para la fórmula del m-estimador como el promedio del valor "e" para todas las palabras que apoyan la hipótesis h. Esta suposición puede tener sentido si las frases escritas de alguna manera son consistentes con las frases de entrenamiento.

  - **Estrategia 2**: hacemos el promedio de los términos calculados para las previas reales, y tomamos eso como estimación para los términos faltantes.

 En ambos casos completamos la sumatoria para llegar a la cantidad de términos deseada. Cuanto mayor es la probabilidad ajustada promedio para una frase obtenida, mejor se considera la predicción.

 Es importante notar que esta probabilidad ajustada se calcula únicamente a los efectos de la evaluación del modelo y se hace luego de que el algoritmo ya encontró el h_MAP. Es decir, no afecta el funcionamiento del modelo, más allá de agregar una pequeña penalización en el rendimiento computacional por los cálculos adicionales.

## La siguente celda contiene el programa principal para probar el predictor

* El valor de la variable **MODO_EVALUACION** permite prender o apagar el modo de evaluación, de manera de poder evitar la penalización de rendimiento por los cálculos adicionales.

In [ ]:
from math import log, inf, exp

# 0: para apagar el modo evaluación
# 1: para prender el modo evaluación
MODO_EVALUACION = 1

def m_estimador(e, n, p):
  return (e + m * p) / (n + m)

def recomendacion_bayesiana(frase):
  cantidad_palabras_distintas = len(P)
  probabilidad_p = 1/cantidad_palabras_distintas

  h_MAP = ""
  p_MAP = -inf

  for h in P.keys():
    prob = log(P[h] / cantidad_total_palabras)
    terminos = []
    for d in frase[-N:]:
      d = normalizar_palabra(d)
      e = PD[h].get(d, 0)
      n = P[h]
      prob_m_estimador = log(m_estimador(e, n, probabilidad_p))
      prob = prob + prob_m_estimador
      terminos.append(prob_m_estimador)

    if prob > p_MAP:
      h_MAP, p_MAP, terminos_MAP = h, prob, terminos


  if MODO_EVALUACION == 1:
    # Modo de evaluación activado
    cant_terminos = len(frase[-N:])

    # Estrategia de evaluación 1
    e_norm = sum(PD[h_MAP].values()) / len(PD[h_MAP])
    n = P[h_MAP]
    m_estimador_norm = log(m_estimador(e_norm, n, probabilidad_p))
    p_MAP_norm = p_MAP + (4 - cant_terminos) * m_estimador_norm
    probabilidades[1][N].append(exp(p_MAP_norm))

    # Estrategia de evaluación 2
    termino_norm = sum(terminos_MAP) / cant_terminos
    p_MAP_norm = p_MAP + (4 - cant_terminos) * termino_norm
    probabilidades[2][N].append(exp(p_MAP_norm))

  return h_MAP


##### LOOP PRINCIPAL #####

print("Ingrese la frase dando ENTER luego de \x1b[3mcada palabra\x1b[0m.")
print("Ingrese sólo ENTER para aceptar la recomendación sugerida, o escriba la siguiente palabra y de ENTER")
print("Ingrese '.' para comenzar con una frase nueva.")
print("Ingrese '..' para terminar el proceso.")

frase = []
palabra_sugerida = ""
while 1:
    palabra = input(">> ")

    if palabra == "..":
      break

    elif palabra == ".":
      print("----- Comenzando frase nueva -----")
      cantidad_total_palabras += entrenar(frase)
      frase = []

    elif palabra == "": # acepta última palabra sugerida
      frase.append(palabra_sugerida)

    else: # escribió una palabra
      frase.append(palabra)

    if frase:
      palabra_sugerida = recomendacion_bayesiana(frase)

      frase_propuesta = frase.copy()
      frase_propuesta.append("\x1b[3m"+ palabra_sugerida +"\x1b[0m")

      print(" ".join(frase_propuesta))





Ingrese la frase dando ENTER luego de cada palabra.
Ingrese sólo ENTER para aceptar la recomendación sugerida, o escriba la siguiente palabra y de ENTER
Ingrese '.' para comenzar con una frase nueva.
Ingrese '..' para terminar el proceso.
>> quiero
quiero que
>> 
quiero que no
>> 
quiero que no se
>> 
quiero que no se lo
>> 
quiero que no se lo que
>> 
quiero que no se lo que no
>> 
quiero que no se lo que no se
>> 
quiero que no se lo que no se lo
>> 
quiero que no se lo que no se lo que
>> .
----- Comenzando frase nueva -----
>> mañana
mañana no
>> es
mañana es un
>> la
mañana es la de
>> final
mañana es la final de
>> del
mañana es la final del mundo
>> 
mañana es la final del mundo que
>> ..


## La siguientes celdas se utilizan para mostrar el promedio de las probabilidades MAP ajustadas para los distintos valores del hiper parámetro N con ambas estrategias de evaluación

In [ ]:
from statistics import mean

print("Probabilidades promedio estrategia de evaluación 1")
for k, v in probabilidades[1].items():
  if len(v) > 0:
    print(f"{k}:{mean(v)}")

Probabilidades promedio estrategia de evaluación 1
1:2.211419522795101e-08


In [ ]:
from statistics import mean

print("Probabilidades promedio estrategia de evaluación 2")
for k, v in probabilidades[2].items():
  if len(v) > 0:
    print(f"{k}:{mean(v)}")

Probabilidades promedio estrategia de evaluación 2
1:4.628090502724307e-06


## Experimentación

Se presentan a continuación los resultados obtenidos obtenidos al entrenar el modelo con un chat de alrededor de 42000 mensajes de un grupo de amigos, con distintos valores de N.

En la primera prueba se comienza una frase con la palabra "quiero", y se acepta la predicción 8 veces seguidas. Luego de eso se utiliza el comando "." para reentrenar el modelo y comenzar la segunda prueba.

En la segunda prueba se forma la frase "mañana es la final del mundo", aceptando la sugerencia cuando corresponda con la palabra deseada.

Para cada valor de N, se muestra la frase obtenida de la primera prueba, así como también el promedio de la probabilidad ajustada para todas las predicciones del modelo.

# N=1
Frase obtenida: "quiero que no se lo que no se lo"  
Estrategia 1: 2.211419522795101e-08  
Estrategia 2: 4.628090502724307e-06

# N=2
Frase obtenida: "quiero que gane el mundial de mayores finales de"  
Estrategia 1: 4.31141515363044e-08  
Estrategia 2: 2.8782745626775494e-07

# N=3
Frase obtenida: "quiero que gane firmar el el el resto de"  
Estrategia 1: 6.275309728061651e-07  
Estrategia 2: 1.418655788852198e-05

# N=4
Frase obtenida: "quiero que gane firmar firmar deslinde deslinde camilo camilo"  
Estrategia 1: 2.742773675870112e-08  
Estrategia 2: 2.4789274118106713e-07

## 4. Conclusión

## 4.1 Resultados

En la Tabla 1, se presenta un resumen de los resultados obtenidos para las distintas pruebas, con los distintos valores de N y ambas estrategias de evaluación.

<table>
  <tr>
    <th>N</th>
    <th>Frase obtenida</th>
    <th>Estrategia 1</th>
    <th>Estrategia 2</th>
  </tr>
  <tr>
    <th>1</th>
    <td>quiero que no se lo que no se lo</td>
    <td>2.211419522795101e-08</td>
    <td>4.628090502724307e-06</td>
  </tr>    
  <tr>
    <th>2</th>
    <td>quiero que gane el mundial de mayores finales de</td>
    <td>4.31141515363044e-08</td>
    <td>2.8782745626775494e-07</td>
  </tr>
  <tr>
    <th>3</th>
    <td>quiero que gane firmar el el el resto de</td>
    <td>6.275309728061651e-07</td>
    <td>1.418655788852198e-05</td>
  </tr>
  <tr>
    <th>4</th>
    <td>quiero que gane firmar firmar deslinde deslinde camilo camilo</td>
    <td>2.742773675870112e-08</td>
    <td>2.4789274118106713e-07</td>
  </tr>
  <caption>Tabla 1 - Calidad de predicciones del modelo para cada valor de N</caption>
</table>

En general se obtuvieron predicciones coherentes al principio, perdiendose un poco el sentido de la frase hacia el final. La mejor calidad de predicciones (según los resultados de las dos estrategias de evaluación) se dió en el caso de N=3, esto puede deberse a que analizar 3 palabras previas es un equilibrio entre considerar pocas palabras y no llegar a tomar todo el contexto, y tomar demasiadas palabras, donde algunas ya no aportan a lo que se quiere predecir.

Al tomar N=1 lo que ocurre es que las probabilidades condicionales tienen el mismo peso en la sumatoria que la probabilidad absoluta. Por lo tanto empiezan a aparecer en las sugerencias muchos monosílabos que tienen una alta frecuencia absoluta en el texto de entrenamiento.

Por otro lado con el valor de N=4 se observa con bastante frecuencia que una misma palabra se sugiere dos veces seguidas. Lo que ocurre es que a medida que se aumenta la ventana de contexto, menor es el peso relativo que tendrá la última palabra en el cálculo de las probabilidades. Por lo tanto, mayor será la probabilidad de que la última h_MAP siga siendo la mejor para este nuevo contexto.


### 4.2 Oportunidades de mejora

- Se podrían obtener mejores resultados entrenando sobre un documento de mayor tamaño, ya que tendría más palabras en el diccionario P y combinaciones de palabras representadas por cada diccionario PD[h] disponibles a la hora de predecir.

- En ocasiones la palabra sugerida por el modelo coincide con la última palabra de la frase. Dado que es muy raro que una palabra aparezca dos veces seguidas en una frase, se podría hacer un ajuste al algoritmo para evitar ese tipo de sugerencia, por ejemplo tomando la segunda palabra con mayor probabilidad en esos casos. Como se mencionó anteriormente, estas situaciones son particularmente notorias a medida que se aumenta el valor de N.

- El diccionario utilizado no contiene palabras con tildes ni nombres propios, lo cual es una limitante para el vocabulario a predecir.